<a href="https://colab.research.google.com/github/hyang0129/2022TSR/blob/main/ROBUST_AUGMENTATIONS_FOR_MULTIVARIATE_TIME_SERIES_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OVERVIEW OF NOTEBOOK

If you are unfamiliar with colab, please try the intro to colab notebook at https://colab.research.google.com/notebooks/intro.ipynb. 

If you are familiar with running code in colab, you can rerun any of the experiments from the paper ROBUST AUGMENTATIONS FOR MULTIVARIATE TIME SERIES CLASSIFICATION by adjusting the values in the hyper parameter section then running all the cells. 

Please note that the first time you run this notebook, it will install python packages and you will encounter the error below. 
```
ImportError: cannot import name 'AggFuncTypeDict' from 'pandas._typing' (/usr/local/lib/python3.7/dist-packages/pandas/_typing.py)
```
Simply restart the runtime using the runtime dropdown menu at the top and run all the code again. I have not figured out why this error occurs, but it only occurs when you first install the requirements.

When you run the notebook, once it finishes a training run for a dataset, model, and seed, it will save the results to a CSV. Right now it will save to the colab instance, which will be wiped after the session is over. 


# SETUP CODE BLOCK

In [1]:
!git clone https://github.com/2022ICMLRAMTSC/CODE
!(cd CODE ; pip install -r requirements.txt -q)


fatal: destination path 'CODE' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.8.1 requires statsmodels<=0.12.1, but you have statsmodels 0.13.1 which is incompatible.


In [2]:
import sys 
sys.path.append('CODE')

In [3]:
%load_ext autoreload


In [4]:
%autoreload

from tsr.datasets.datasetmanager import DatasetManager
from tsr.datasets import TEP_DatasetManager
from tsr.config import Config
import gdown
import pandas as pd
import numpy as np
from sklearn import preprocessing
from tqdm.autonotebook import tqdm
import tensorflow as tf
from loguru import logger
import pyreadr as py
from sklearn.model_selection import train_test_split, KFold
from tsr.utils import shell_exec
from tsr.tpusetup import connect_to_tpu
from tsr.datasets import UAE_DatasetManager
from tsr.models.attention import ConvMHSA



CODE/tsr/datasets/ngafid.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# HYPER PARAMETERS

In [5]:

aug_version = 'None' # Any aug code in the paper, so A through G and None  
save_path = '' # empty will just save it to the colab instance 
model_name = 'InceptionTime' # InceptionTime, SimpleRNN, or SimpleMHSA


start = 0 # Initial Seed (for skipping seeds if we are parallelizing the experiments)
repeats = 1 # Total Number of Seeds to Iterate Through
verbose = True # Print more stuff and progress bars
start_at_dataset = 0 # Leave at zero if you want to run through all the datasets
                    # to skip ahead and try for a specific dataset, see the prepare dataset section 
                    # for the index number of each dataset

In [6]:
# Aug versions were originally coded as numbers. 

aug_version_dict = {
    'None' : -1, 
    'A' : 1, 
    'B' : 2, 
    'C' : 3, 
    'D' : 4, 
    'E' : 5, 
    'F' : 6, 
    'G' : 8,
}
aug_version = aug_version_dict[aug_version]

# PREPARE DATASET

In [7]:

config = Config.get_standard_config('TEP.yaml')
config.as_attr_dict()
dsm = UAE_DatasetManager(config)



2022-01-27 03:19:40.519 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for hyperparameters config group
2022-01-27 03:19:40.531 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for model config group
2022-01-27 03:19:40.532 | DEBUG    | tsr.config:__init__:62 - {'model': {'architecture_name': 'conv_mhsa', 'input_shape': [500, 52], 'conv_filters': [128, 128, 512], 'kernel': [7, 7, 7], 'strides': [1, 2, 2], 'num_heads': 8, 'd_model': 512, 'dff': 512}}
2022-01-27 03:19:40.544 | INFO     | tsr.config:collect_sub_configs:138 - Added Subconfig for model from CODE/tsr/configs/model/conv_mhsa_tep.yaml
2022-01-27 03:19:40.548 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for augment config group
2022-01-27 03:19:40.555 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for environment config group
2022-01-27 03:19:40.566 | DEBUG    | tsr.config:__init__:62 - {'hyperparameters': {'NFOLD': 5

In [13]:
for i in np.array(range(len(list(dsm.directories.keys())))):

    print(i, list(dsm.directories.keys())[i])

0 ArticularyWordRecognition
1 AtrialFibrillation
2 BasicMotions
3 Cricket
4 DuckDuckGeese
5 ERing
6 EigenWorms
7 Epilepsy
8 EthanolConcentration
9 FaceDetection
10 FingerMovements
11 HandMovementDirection
12 Handwriting
13 Heartbeat
14 LSST
15 Libras
16 MotorImagery
17 NATOPS
18 PEMS-SF
19 PenDigits
20 PhonemeSpectra
21 RacketSports
22 SelfRegulationSCP1
23 SelfRegulationSCP2
24 StandWalkJump
25 UWaveGestureLibrary


# CONNECT TPU

In [8]:
strategy = connect_to_tpu()

TPU address is grpc://10.36.41.234:8470
Running on TPU  grpc://10.36.41.234:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.36.41.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.36.41.234:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# TRAINING

In [11]:
from tsr.models.inceptiontime import Classifier_INCEPTION
from tsr.methods.augmentation import get_augs 
import os 
from tqdm.notebook import tqdm
from tsr.models.attention.simplemhsa import SimpleMHSA
from tsr.models.rnn.simplernn import SimpleRNN




for seed in tqdm(range(start, repeats)):

    for i in tqdm(np.array(range(len(list(dsm.directories.keys()))))[start_at_dataset:]):

        dataset_name = list(dsm.directories.keys())[i]
        save_to_file = os.path.join(save_path, '_'.join([dataset_name, model_name, 'SEED%02.f' % seed, 'AUG%02.f' % aug_version, '.csv']))
        
        if not os.path.exists(save_to_file): 
            ds, test_ds, shape, num_classes = dsm.get_datasets_as_tf(list(dsm.directories.keys())[i], resample_seed = seed)

            if shape[1] < 513:

                with strategy.scope():
                    

                    if model_name == 'SimpleMHSA': 
                        model = SimpleMHSA(input_shape=shape[1:], num_class=num_classes)    
                    elif model_name == 'InceptionTime':
                        model = Classifier_INCEPTION(input_shape = (shape[1:]), nb_classes=num_classes)
                        model = model.model # this is just how its done. Code for 
                        # inceptiontime was copied from https://github.com/hfawaz/InceptionTime with no edits
                        # except to get it to work with tfkeras. 
                    elif model_name == 'SimpleRNN':
                        model = SimpleRNN(input_shape=shape[1:], num_class=num_classes)   
                    else:
                        raise KeyError('Check your model name')

                    if verbose:
                        model.summary()

                    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
                                metrics = [
                                            tf.keras.metrics.SparseCategoricalAccuracy()
                                ], 
                                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False))
                    
                    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10,
                                                                min_lr=0.0001)
                    

                    early_stop = tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss', min_delta=0, patience=100, verbose=0,
                        mode='auto', baseline=None, restore_best_weights=True
                    )

                    print(dataset_name, model_name)

                    history = model.fit(
                        ds.map(get_augs(SHAPE = shape , BATCH_SIZE = 64, DO_PROB=None, element_prob = None, version = aug_version)), 
                        steps_per_epoch = max(int(shape[0]/64 +1), 100), 
                        epochs = 1500, 
                        validation_data = test_ds,
                        callbacks = [
                                    reduce_lr, early_stop
                        ],
                        verbose = verbose, 
                    )


                results = pd.DataFrame(history.history)

                dataset_name = list(dsm.directories.keys())[i]

                results['dataset_name'] = dataset_name
                results['model'] = model_name
                results['epoch'] = results.index
                results['do_prob'] = DO_PROB
                results['element_prob'] = element_prob
                results['aug_version'] = aug_version 
                results['seed'] = seed 
                results.to_csv(save_to_file) 

# there is a strong correlation between the quality of the code and time remaining till the deadline. 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 144, 9)]     0                                            
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 144, 32)      288         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_6 (MaxPooling1D)  (None, 144, 9)       0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 144, 32)      40960       conv1d_32[0][0]                  
____________________________________________________________________________________________

KeyboardInterrupt: ignored